## Capstone Data

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decision are:
* The total number of crimes commited in each of the borough during the last year.
* The most common venues in each of the neighborhood in the safest borough selected.

Following data sources will be needed to extract/generate the required information:

- [**Part 1**: Preprocessing a real world data set from Kaggle showing the London Crimes from 2008 to 2016](#part1):  A dataset consisting of the crime statistics of each borough in London obtained from Kaggle
- [**Part 2**: Scraping additional information of the different Boroughs in London from a Wikipedia page.](#part2): More information regarding the boroughs of London is scraped using the Beautifulsoup library
- [**Part 3**: Creating a new dataset of the Neighborhoods of the safest borough in London and generating their co-ordinates.](#part3): Co-ordinate of neighborhood will be obtained using **Google Maps API or Foursquare geocoding**

### Part 1: Preprocessing a real world data set from Kaggle showing the London Crimes from 2008 to 2016<a name="part1"></a>


####  London Crime Data 

About this file

- lsoa_code: code for Lower Super Output Area in Greater London.
- borough: Common name for London borough.
- major_category: High level categorization of crime
- minor_category: Low level categorization of crime within major category.
- value: monthly reported count of categorical crime in given borough
- year: Year of reported counts, 2008-2016
- month: Month of reported counts, 1-12

Data set URL: https://www.kaggle.com/jboysen/london-crime

In [ ]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup # library for web scrapping  

!conda install -c conda-forge geocoder --yes
import geocoder

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: | 

#### Define Foursquare Credentials and Version
Make sure that you have created a Foursquare developer account and have your credentials handy

In [ ]:
CLIENT_ID = 'LJB534MNVKNBS3M1IWFRJIFYSIVCNVBB3WHCIX0VJIYDMHFV' # your Foursquare ID
CLIENT_SECRET = 'INXBHAGUZIBFC15QIOG3NT3S42B1EKBFQ2U3VPTBBS5EERVM' # your Foursquare Secret

VERSION = '20180604'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Read in the dataset

In [ ]:
df = pd.read_csv("london_crime_by_lsoa.csv")

In [ ]:
df.head()

#### Accessing the most recent crime rates (2016)

In [ ]:
df.drop(df.index[df['year'] != 2016], inplace = True)
df = df[df.value != 0]
df = df.reset_index(drop=True)
df.shape()

In [ ]:
df.head()

#### Change the column names 

In [ ]:
df.columns = ['LSOA_Code', 'Borough','Major_Category','Minor_Category','No_of_Crimes','Year','Month']
df.head()

In [ ]:
df.info()

#### Total number of crimes in each Borough

In [ ]:
df['Borough'].value_counts()

#### The total crimes per major category

In [ ]:
df['Major_Category'].value_counts()

#### Pivoting the table to view the number of crimes for each major category in each Borough 

In [ ]:
London_crime = pd.pivot_table(df,values=['No_of_Crimes'],
                               index=['Borough'],
                               columns=['Major_Category'],
                               aggfunc=np.sum,fill_value=0)
London_crime.head()

In [ ]:
London_crime.reset_index(inplace = True)
London_crime['Total'] = London_crime.sum(axis=1)
London_crime.head(33)

#### Removing the multi index to merge

In [ ]:
London_crime.columns = London_crime.columns.map(''.join)
London_crime.head()

In [ ]:
London_crime.columns = ['Borough','Burglary', 'Criminal Damage','Drugs','Other Notifiable Offences',
                        'Robbery','Theft and Handling','Violence Against the Person','Total']
London_crime.head()

In [ ]:
London_crime.shape

### Part 2: Scraping additional information of the different Boroughs in London from a Wikipedia page <a name="part2"></a>
 
**Using Beautiful soup to scrap the latitude and longitiude of the boroughs in London**

URL: https://en.wikipedia.org/wiki/List_of_London_boroughs

In [ ]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_London_boroughs'
raw_wikipedia_page= requests.get(wikipedia_link).text
soup = BeautifulSoup(raw_wikipedia_page,'xml')
print(soup.prettify()

In [ ]:
table = soup.find_all('table', {'class':'wikitable sortable'})
print(table)

In [ ]:
London_table = pd.read_html(str(table[0]), index_col=None, header=0)[0]
London_table.head()

#### The second table on the site contains the addition Borough i.e. City of London

In [ ]:
London_table1 = pd.read_html(str(table[1]), index_col=None, header=0)[0]
London_table1.columns = ['Borough','Inner','Status','Local authority','Political control',
                         'Headquarters','Area (sq mi)','Population (2013 est)[1]','Co-ordinates','Nr. in map']

London_table1

In [ ]:
London_table = London_table.append(London_table1, ignore_index = True) 
London_table.head()

In [ ]:
# Check the last row
London_table.tail()

#### Info table

In [ ]:
London_table.info()

In [ ]:
# remove unecessary data
London_table = London_table.replace('note 1','', regex=True) 
London_table = London_table.replace('note 2','', regex=True) 
London_table = London_table.replace('note 3','', regex=True) 
London_table = London_table.replace('note 4','', regex=True) 
London_table = London_table.replace('note 5','', regex=True) 
London_table.head()

In [ ]:
type(London_table)

In [ ]:
London_table.shape

In [ ]:
# Check if data is match
set(df.Borough) - set(London_table.Borough)

In [ ]:
# find index that don't match
print("The index of first borough is",London_table.index[London_table['Borough'] == 'Barking and Dagenham []'].tolist())
print("The index of second borough is",London_table.index[London_table['Borough'] == 'Greenwich []'].tolist())
print("The index of third borough is",London_table.index[London_table['Borough'] == 'Hammersmith and Fulham []'].tolist())

#### Changing the Borough names to match the other data frame

In [ ]:
London_table.iloc[0,0] = 'Barking and Dagenham'
London_table.iloc[9,0] = 'Greenwich'
London_table.iloc[11,0] = 'Hammersmith and Fulham'

#### Check if the Borough names in both data sets match

In [ ]:
set(df.Borough) - set(London_table.Borough)

#### Combine both the data frames together


In [ ]:
Ld_crime = pd.merge(London_crime, London_table, on='Borough')
Ld_crime.head(10)

In [ ]:
Ld_crime.shape

In [ ]:
set(df.Borough) - set(Ld_crime.Borough)

In [ ]:
list(Ld_crime)

In [ ]:
columnsTitles = ['Borough','Local authority','Political control','Headquarters',
                 'Area (sq mi)','Population (2013 est)[1]',
                 'Inner','Status',
                 'Burglary','Criminal Damage','Drugs','Other Notifiable Offences',
                 'Robbery','Theft and Handling','Violence Against the Person','Total','Co-ordinates']

Ld_crime = Ld_crime.reindex(columns=columnsTitles)

Ld_crime = Ld_crime[['Borough','Local authority','Political control','Headquarters',
                 'Area (sq mi)','Population (2013 est)[1]','Co-ordinates',
                 'Burglary','Criminal Damage','Drugs','Other Notifiable Offences',
                 'Robbery','Theft and Handling','Violence Against the Person','Total']]

Ld_crime.head()